- 콘텐츠 기반 필터링: 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면, 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천
    * ex) '인셉션' → 인셉션의 장르인 액션, 공상과학으로 높은 평점을 받은 영화를 추천하거나, '인셉션'의 감독인 크리스토퍼 놀란의 다른 영화를 추천하는 방식.

# 1) 데이터 로딩 및 가공

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


주요 column; id, title, genres, vote_average, vote_count, popularity, keywords, overview

In [2]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [3]:
pd.set_option('max_colwidth', 200)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name..."


In [4]:
from ast import literal_eval

# dictionary → list
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [5]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [ y['name'] for y in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations..."


# 2) 장르 속성
- 장르 column의 유사도를 비교한 뒤 그 중 높은 평점을 가지는 영화 추천
- 장르를 문자열로 변경한 뒤 이를 CountVectorizer로 피처벡터화한 행렬 데이터값을 코사인 유사도로 비교
- 장르 유사도가 높은 영화 중에 평점이 높은 순서로 영화를 추천함

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))

# min_df: 전체 문서에서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터 (정수일 경우 횟수, 부동소수점일 경우 비중)
# ngram_range: 튜플 형태로 (범위 최솟값, 범위 최댓값) 지정.
# ex) (1,1): 토큰화된 단어를 1개씩 피처로 추출, (1,2): 토큰화된 단어를 1개씩 그리고 순서대로 2개씩 묶어서 피처로 추출
count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [8]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:2])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]]


In [9]:
# 장르 유사도에 따라 영화를 추천하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 movies_df Dataframe에서 'title' 컬럼이 입력된 title_name 값인 Dataframe 추출
    title_movie = df[df['title'] == title_name]
    
    # title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    sim_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력. top_n은 2차원 데이터임
    # dataFrame에서 index로 사용하기 위해 1차원 array로 변경
    print(sim_indexes)
    sim_indexes = sim_indexes.reshape(-1)
    
    return df.iloc[sim_indexes]

In [10]:
sim_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
sim_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- 평점이 0인 것도 있고, Light Sleeper, Mi America, Kids 등 적절하지 못한 결과도 섞여 있음
- 더 많은 후보군을 선정한 후 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경

In [11]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
    - v: 개별 영화에 평점을 투표한 횟수 (vote_count)
    - m: 평점을 부여하기 위한 최소 투표 횟수 (m값을 높이면 평점 투표 횟수가 더 많은 영화에 많은 가중치를 부여함, 여기서는 상위 60%로 설정)
    - R: 개별 영화에 대한 평균 평점 (vote_average)
    - C: 전체 영화에 대한 평균 평점 (movies_df['vote_average'].mean())

In [12]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( ( (v/(v+m)) * R ) + ( (m/(m+v)) * C ) )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


- 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤에 weighted_vote 칼럼 값이 높은 순으로 top_n만큼 추출하는 방식

In [13]:
def find_sim_movie_new(df, sorted_ind, title_name, top_n = 10):

    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    sim_indexes = sorted_ind[title_index, :(top_n*2)]
    sim_indexes = sim_indexes.reshape(-1)
    sim_indexes = sim_indexes[sim_indexes != title_index]
    
    return df.iloc[sim_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [14]:
sim_movies = find_sim_movie_new(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
sim_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


# 3) Overview (using TF-IDF)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
# 전처리 - 결측치가 있는 항목은 모두 제거
print(movies_df.shape)
movies_df = movies_df[movies_df['overview'].notnull()].reset_index(drop=True)
print(movies_df.shape)

(4803, 10)
(4800, 10)


In [18]:
# 불용어: 유의미하지 않은 단어 토큰 제거
tfidf = TfidfVectorizer(stop_words = 'english')

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(movies_df['overview'])
print(tfidf_matrix.shape)

(4800, 20978)


In [19]:
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [30]:
genre_sim_sorted_ind = cosine_matrix.argsort()[:, ::-1]
genre_sim_sorted_ind[:2]

array([[   0, 3603, 2130, ..., 3159, 3161, 2399],
       [   1, 2542, 3094, ..., 2936, 2934,    0]], dtype=int64)

In [32]:
sim_movies = find_sim_movie_new(movies_df, genre_sim_sorted_ind,'The Godfather', 10)
sim_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2730,The Godfather: Part II,8.3,8.079586
444,Road to Perdition,7.3,6.991032
867,The Godfather: Part III,7.1,6.905293
3895,Sinister,6.7,6.584202
3163,Loose Cannons,7.0,6.311598
3782,Joe,6.5,6.286814
3726,Easy Money,6.5,6.151482
3622,Made,6.3,6.119054
3124,Eulogy,6.4,6.118065
2001,The Crew,6.4,6.101054
